In [1]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaLLM

model = OllamaLLM(model = "deepseek-r1:1.5b",base_url = "http://localhost:11434",temperature=0.0)

file = "OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=file)
data= loader.load()

In [2]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings

# 替换为你实际的模型文件夹路径
local_model_path = "all-MiniLM-L6-v2" 

# 创建嵌入模型
embeddings = HuggingFaceEmbeddings(
    model_name=local_model_path,
    model_kwargs={"device": "cuda"}  # 如需用GPU，改为"cuda"，确保CUDA环境配置正确
)
# 创建向量存储索引
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings  # 必须指定嵌入模型
).from_loaders([loader])

C:\Users\33249\AppData\Local\Temp\ipykernel_21860\948672960.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\33249\anaconda3\envs\xhd\Lib\site-packages\sklearn\utils\_param_validation.py:11: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy.sparse import csr_matrix, issparse


c:\Users\33249\anaconda3\envs\xhd\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [3]:
# 创建一个RetrievalQA对象，用于从向量存储中检索答案 
# 参数说明：   llm=model: 使用指定的语言模型（llm）作为生成答案的基础模型   
# chain_type="stuff": 指定使用"stuff"类型的链，这是一种常见的问答链类型   
# retriever=index.vectorstore.as_retriever(): 从向量存储中获取检索器，用于在问答过程中检索相关信息
# verbose=True: 打开详细输出，以便在执行过程中查看更多信息
# chain_type_kwargs={"document_separator":"<<<<>>>>"}: 设置链类型的关键字参数，
# 其中"document_separator"参数用于指定文档分隔符，以便在处理多个文档时进行分隔

qa = RetrievalQA.from_chain_type(llm=model,chain_type="stuff", 
                    retriever=index.vectorstore.as_retriever(),
                    verbose=True,
                    chain_type_kwargs={"document_separator":"<<<<>>>>"})

In [4]:
#  定义一个名为examples的列表，用于存储示例问题及其答案
examples = [ 
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

In [5]:
# from langchain.evaluation.qa import QAGenerateChain
# example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())
#new_examples = example_gen_chain.apply_and_parse([{"doc": t} for t in data[:5]])

# 使用OpenAI的ChatOpenAI 来进行问答生成，因为ChatOpenAI模型的输出格式默认的是RegexParser，可以解析出问题和答案
# 但是在我们使用本地Ollama模型时，输出格式与ChatOpenAI模型的输出格式不匹配，所以需要自定义输出解析器

使用OpenAI的ChatOpenAI 来进行问答生成，因为ChatOpenAI模型的输出格式默认的是RegexParser，可以解析出问题和答案

但是在我们使用本地Ollama模型时，输出格式与ChatOpenAI模型的输出格式不匹配，所以需要自定义输出解析器

1. 输出解析器不匹配

当使用非 OpenAI 模型（如 ChatOllama）时，模型输出格式可能与 RegexParser 预期不符（例如包含思考过程、中英文混合或格式混乱），导致解析失败。

2. 未显式配置提示模板

QAGenerateChain.from_llm(ChatOpenAI()) 隐式使用了适合 OpenAI 的提示模板，而你使用其他模型时，需显式定义 强制结构化输出的提示模板，否则模型可能自由发挥。

In [6]:
from langchain.prompts import PromptTemplate
from langchain.evaluation.qa import QAGenerateChain
from langchain.chat_models import ChatOllama
from langchain.schema import BaseOutputParser
import csv

# ----------------------
# 1. 自定义提示模板（保持不变）
# ----------------------
# #  定义一个提示模板，用于生成基于产品描述的英文问答对
prompt_template = """ 
Given the following English product description, generate one English Question-Answer pair (QA pair) that:
1. Focuses on specific data in Specs, Construction, or Features (e.g., weight, material, dimensions).
2. The question ends with a question mark (?) and the answer is an exact match from the description.

Strict format:
Question: <specific English question>
Answer: <exact English answer>

Product description:
{doc}
"""
#  使用定义的模板创建一个PromptTemplate对象，其中input_variables指定了模板中的变量
prompt = PromptTemplate(input_variables=["doc"], template=prompt_template) 

# ----------------------
# 2. 输出解析器（保持不变）
# ----------------------
class QAOutputParser(BaseOutputParser):
    def parse(self, text):
        cleaned_text = "\n".join([line for line in text.split('\n') if "Question:" in line or "Answer:" in line])
        question = ""
        answer = ""
        for line in cleaned_text.split('\n'):
            line = line.strip()
            if line.startswith("Question:"):
                question = line[len("Question:"):].strip()
                if not question.endswith('?'):
                    question += '?'
            elif line.startswith("Answer:"):
                answer = line[len("Answer:"):].strip()
        return {"question": question, "answer": answer}

# ----------------------
# 3. 初始化模型（保持不变）
# ----------------------
llm = ChatOllama(
    model="deepseek-r1:1.5b",
    base_url="http://localhost:11434",
    temperature=0.0 #  设置温度参数为0.0，通常用于控制生成文本的随机性，温度越低生成的文本越确定性
)
#  创建一个QAGenerateChain对象，用于生成问答链，传入语言模型llm和提示prompt
qa_chain = QAGenerateChain(llm=llm, prompt=prompt) 
#  设置输出解析器，用于解析模型的输出结果，并设置输出解析器为QAOutputParser对象
qa_chain.output_parser = QAOutputParser()

# ----------------------
# 4. 加载数据并筛选部分数据
# ----------------------
def load_new_examples(csv_path, max_items=5):  # max_items=要处理的最大数量（默认前5条）
    data = []
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for i, row in enumerate(reader, 1):
            if i > max_items:  # 超过max_items时停止读取
                break
            doc = row['description'].replace('"', '').replace('\n', ' ')
            data.append({"doc": doc})
    return data

# ----------------------
# 5. 处理部分数据（示例：处理前5条）
# ----------------------
csv_path = "OutdoorClothingCatalog_1000.csv"
new_examples = load_new_examples(csv_path, max_items=5)  # 仅处理前5条

converted_examples = [] #  初始化一个空列表，用于存储转换后的示例数据
for idx, item in enumerate(new_examples, 1):
    try:
        result = qa_chain.run(item)
        print(f"第{idx}条QA对:")
        print(f"问题: {result['question']}")
        print(f"答案: {result['answer']}\n")
        converted_examples.append({"query": result['question'], "answer": result['answer']})
    except Exception as e:
        print(f"第{idx}条处理失败: {str(e)}\n")

C:\Users\33249\AppData\Local\Temp\ipykernel_21860\708442225.py:47: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(
C:\Users\33249\AppData\Local\Temp\ipykernel_21860\708442225.py:80: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(item)


第1条QA对:
问题: What is the approximate weight of this ultracomfortable lace-to-toe Oxford shoe?
答案: Approximate weight: 1 lb. 1 oz.

第2条QA对:
问题: What are the dimensions of the Small mat?
答案: 18 x 28 inches

第3条QA对:
问题: 
答案: 

第4条QA对:
问题: What is the UPF rating of this tankini top?
答案: The tankini top has an UPF 50+ rating, which is the highest rated sun protection possible.

第5条QA对:
问题: What material does our TEK O2 technology use for waterproofing?
答案: Three-layer shell delivers waterproof protection.



In [7]:
new_examples

[{'doc': 'This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on.   Size & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size.   Specs: Approx. weight: 1 lb.1 oz. per pair.   Construction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported.   Questions? Please contact us for any inquiries.'},
 {'doc': "Protect your floors from spills and splashing with our ultradurable recycled Waterhog dog mat made right here in the USA.   Specs Small - Dimensions: 18 x 28.  Medium - Dimensions: 22.5 x 34.5.  Why We Love It Mother nature, wet shoes and muddy paws ha

In [8]:
converted_examples

[{'query': 'What is the approximate weight of this ultracomfortable lace-to-toe Oxford shoe?',
  'answer': 'Approximate weight: 1 lb. 1 oz.'},
 {'query': 'What are the dimensions of the Small mat?',
  'answer': '18 x 28 inches'},
 {'query': '', 'answer': ''},
 {'query': 'What is the UPF rating of this tankini top?',
  'answer': 'The tankini top has an UPF 50+ rating, which is the highest rated sun protection possible.'},
 {'query': 'What material does our TEK O2 technology use for waterproofing?',
  'answer': 'Three-layer shell delivers waterproof protection.'}]

In [9]:
# new_examples 列表里的元素是只有 doc 键的，而 converted_examples 才是转换后有 query 和 answer 键的。
# 如果是将 new_examples 直接添加到 examples 列表中，可能会导致后续出现examples列表里有些元素只有 doc 键而没有 query 和 answer 键，
# 这样会导致后续的处理出现问题。所以我们将 new_examples 里的元素转换成 query 和 answer 键的形式，用con

examples += converted_examples #  将转换后的示例列表 converted_examples 添加到现有的示例列表 examples 中

In [10]:
# 检查合并后的数据格式
for example in examples:
    if "query" not in example:
        print("发现格式不正确的数据，缺少 'query' 键。")

跨 LLM 生成结构化 QA 的通用公式

提示模板决定输出格式：

无论使用什么 LLM，必须通过提示模板强制输出 {"query": "...", "answer": "..."} 或 Question:/Answer: 格式，这是解析器工作的前提。

温度和停止参数：

temperature=0.7（非 0.0）让模型生成更自然的结构化内容，避免过于机械或简略。

添加 stop=["\n", "思考过程"] 阻止模型输出无关文本。

解析器适配：

若默认 RegexParser 不工作，自定义解析器直接提取 JSON 字段（比正则更鲁棒）：

通过以上步骤，无论使用 ChatOpenAI 还是其他 LLM（如 ChatOllama），都能像示例一样生成结构化问答对。核心在于提示模板强制格式和解析器匹配输出结构，而非依赖特定模型的默认行为。

In [11]:
import langchain
langchain.debug = True #  打开调试模式，查看详细的调试信息

qa.run(examples[0]["query"]) #  运行问答系统，传入第一个示例的查询

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit sits farthest from the body. \n\nFabric & Care \nIn the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features \nRelaxed fit top with raglan sleeves and rounded hem. \nPull-on pants have 

'<think>\nOkay, so I need to figure out if the Cozy Comfort Pullover Set has side pockets. Let me start by reading through the context provided.\n\nFirst, looking at the description for the Cozy Cuddles Knit Pullover Set, it mentions that the top has raglan sleeves and rounded hem with relaxed fit options. It also says the pants have a wide elastic waistband and drawstring, with side pockets and a modern slim leg. Importantly, it lists "Imported." But I don\'t see any mention of side pockets.\n\nThen, for the Cozy Comfort Pullover Set, which is the second name given, it\'s described as having relaxed fit tops with raglan sleeves and rounded hem. It also mentions that the top has side pockets and a modern slim leg. The fabric care details are about the softest blend of 63% polyester, etc.\n\nSo, in both descriptions, the pullover sets mention side pockets but don\'t explicitly state they have them. They talk about relaxed fit tops with raglan sleeves and rounded hem, which suggests that

通过调试信息的上下文可以知道，这个上下文是根据问题检索到的几个文档快内容组成的。  
所以，在做问答时，往往当返回错误的结果时，不一定是语言模型本身出现了问题。  
实际上，可能是在检索的步骤中出现了问题。  
同时，我们也可以在调试信息中看到问答链在底层使用的prompt、还有token消耗、模型名称等详细信息。

与创建示例数据类似，我们也可以通过手动操作的方式来评估我们的链、示例等

In [12]:
langchain.debug = False #  关闭调试模式，恢复默认设置

In [13]:
print(examples)

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'answer': 'Yes'}, {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?', 'answer': 'The DownTek collection'}, {'query': 'What is the approximate weight of this ultracomfortable lace-to-toe Oxford shoe?', 'answer': 'Approximate weight: 1 lb. 1 oz.'}, {'query': 'What are the dimensions of the Small mat?', 'answer': '18 x 28 inches'}, {'query': '', 'answer': ''}, {'query': 'What is the UPF rating of this tankini top?', 'answer': 'The tankini top has an UPF 50+ rating, which is the highest rated sun protection possible.'}, {'query': 'What material does our TEK O2 technology use for waterproofing?', 'answer': 'Three-layer shell delivers waterproof protection.'}]


In [14]:
predictions = qa.batch(examples) #  批量运行问答系统，传入示例列表 examples



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [24]:
from langchain.evaluation.qa import QAEvalChain

llm = ChatOllama(model="deepseek-r1:1.5b", base_url="http://localhost:11434", temperature=0.0) #  初始化语言模型
eval_chain = QAEvalChain.from_llm(llm) #  创建问答评估链


"""在 QAEvalChain.evaluate 方法里，代码会遍历 examples 列表，\
    并且使用相同的索引去访问 predictions 列表里的元素。\
    要是 predictions 列表的长度小于 examples 列表的长度，就会出现索引越界的错误"""


# 检查 examples 和 predictions 列表的长度是否一致
if len(examples) != len(predictions):
    print(f"警告：examples 列表长度 ({len(examples)}) 与 predictions 列表长度 ({len(predictions)}) 不一致。")
    # 可以根据实际情况进行处理，例如截断较长的列表
    min_length = min(len(examples), len(predictions))
    examples = examples[:min_length]
    predictions = predictions[:min_length]

# 评估模型的预测结果，传入示例列表 examples 和预测结果 predictions
graded_outputs= eval_chain.evaluate(examples, predictions)

#  打印评估结果
print(graded_outputs)

[{'results': "<think>\nAlright, let's break this down. The user provided a question about whether the Cozy Comfort Pullover Set has side pockets. They also included examples of other pullover sets with similar features.\n\nFirst, I need to check if both Cozy Comfort and Cozy Cuddles have side pockets. From the context, it seems that both are described as having side pockets in their features. The first set mentions raglan sleeves and rounded hem, while the second one lists a wide elastic waistband and side pockets. \n\nSince both sets are mentioned to have side pockets, I can conclude that the answer is correct.\n</think>\n\nYes, both Cozy Comfort Pullover Set and Cozy Cuddles Knit Pullover Set have side pockets.\n\nTRUE ANSWER: Yes\nGRADE: CORRECT"}, {'results': "<think>\nAlright, let me break down how I approached solving this problem. \n\nFirst, I read through all the provided context about different jackets and their collections. Each jacket's name was listed along with its descrip

In [25]:
# 打印评估结果，键的设置需要观察graded_outputs的结构来决定
#不同模型可能会有不同的输出格式，所以需要根据实际情况来调整键的设置

for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: <think>
Okay, so I need to figure out if the Cozy Comfort Pullover Set has side pockets. Let me start by reading through the context provided.

First, looking at the description for the Cozy Cuddles Knit Pullover Set, it mentions that the top has raglan sleeves and rounded hem with relaxed fit options. It also says the pants have a wide elastic waistband and drawstring, with side pockets and a modern slim leg. Importantly, it lists "Imported." But I don't see any mention of side pockets.

Then, for the Cozy Comfort Pullover Set, which is the second item, it's described as having relaxed fit tops with raglan sleeves and rounded hem. It also mentions that the top has side pockets and a modern slim leg. The additional features section includes "Bonded construction insulates for extra warmth" and other details about the shirt, including collars and hem adjustments.

So, puttin

这段代码用于打印每个示例的问题、真实答案、预测答案和预测等级，但存在一些潜在的缺陷，以下为问题详情：
1. 键存在性问题
代码假设 predictions[i] 字典中一定存在 'query'、'answer' 和 'result' 键，graded_outputs[i] 字典中一定存在 'results' 键。如果这些键不存在，代码会抛出 KeyError 异常。例如，如果 predictions 列表中的某个元素没有 'query' 键，执行 print("Question: " + predictions[i]['query']) 时就会出错。
2. 索引越界问题
代码假设 examples、predictions 和 graded_outputs 列表的长度是相同的，并且在遍历 examples 列表时，使用相同的索引 i 访问 predictions 和 graded_outputs 列表。如果这三个列表的长度不一致，会导致索引越界错误。尽管前面代码中已经对 examples 和 predictions 进行了长度检查和处理，但没有对 graded_outputs 进行同样的处理。
3. 异常处理不足
代码没有对可能出现的异常进行足够的处理。例如，在拼接字符串时，如果 predictions[i]['query'] 或其他键对应的值不是字符串类型，会抛出 TypeError 异常。
4. 代码复用性和可维护性
这段代码将打印逻辑硬编码在循环中，缺乏复用性。如果需要修改打印格式或添加额外的信息，需要直接修改循环内的代码，不利于代码的维护和扩展。

In [27]:
def print_evaluation_result(i, eg, predictions, graded_outputs):
    print(f"Example {i}:")
    try:
        question = predictions[i].get('query', '数据格式错误，缺少 \'query\' 键')
        print(f"Question: {question}")

        real_answer = predictions[i].get('answer', '数据格式错误，缺少 \'answer\' 键')
        print(f"Real Answer: {real_answer}")

        predicted_answer = predictions[i].get('result', '数据格式错误，缺少 \'result\' 键')
        print(f"Predicted Answer: {predicted_answer}")

        predicted_grade = graded_outputs[i].get('results', '数据格式错误，缺少 \'results\' 键')
        print(f"Predicted Grade: {predicted_grade}")
    except IndexError:
        print("索引越界错误，请检查 examples、predictions 和 graded_outputs 列表的长度是否一致。")
    except TypeError:
        print("类型错误，某个键对应的值不是字符串类型。")
    print()


for i, eg in enumerate(examples):
    print_evaluation_result(i, eg, predictions, graded_outputs)

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: <think>
Okay, so I need to figure out if the Cozy Comfort Pullover Set has side pockets. Let me start by reading through the context provided.

First, looking at the description for the Cozy Cuddles Knit Pullover Set, it mentions that the top has raglan sleeves and rounded hem with relaxed fit options. It also says the pants have a wide elastic waistband and drawstring, with side pockets and a modern slim leg. Importantly, it lists "Imported." But I don't see any mention of side pockets.

Then, for the Cozy Comfort Pullover Set, which is the second item, it's described as having relaxed fit tops with raglan sleeves and rounded hem. It also mentions that the top has side pockets and a modern slim leg. The additional features section includes "Bonded construction insulates for extra warmth" and other details about the shirt, including collars and hem adjustments.

So, puttin